### Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.








In [1]:
#Первый вариант пришедший в голову

import pandas as pd

ratings = pd.read_csv('ratings.csv')
ids = (
    pd.DataFrame(pd.value_counts(ratings['userId']))
       .reset_index()
       .rename(columns = {'userId': 'count', 'index': 'userId'}, inplace = False)
      )

final = (
    ids.merge(
    pd.DataFrame(
    ratings.groupby('userId').max()['timestamp'] - ratings.groupby('userId').min()['timestamp'])
                   .reset_index(), how = 'left')
    .rename(columns = {'timestamp': 'lifetime'}, inplace = False)
        )
final.loc[final['count'] > 100]

,userId,count,lifetime
0,547,2391,501810535
1,564,1868,1516358
2,624,1735,457492507
3,15,1700,471393496
4,73,1610,220585493
...,...,...,...
253,217,104,981
254,36,104,5949290
255,623,103,3102
256,257,103,111934874


In [2]:
#Второй вариант когда я вспомнид про функцию count() - это можно считать в одну строчку?:)

final = (
    ratings
    .groupby('userId')
    .count()
    .merge((ratings.groupby('userId').max() - ratings.groupby('userId').min())
           .rename(columns = {'timestamp': 'lifetime'}, inplace = False), on='userId', how = 'left')
    .sort_values(by=['movieId_x'], ascending=False)
        )

final.loc[final.movieId_x > 100, 'lifetime']


userId
547    501810535
564      1516358
624    457492507
15     471393496
73     220585493
         ...    
217          981
36       5949290
623         3102
257    111934874
67          4136
Name: lifetime, Length: 258, dtype: int64

### Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:

<ul>
<li>таблицу с тремя типами выручки для каждого client_id без указания адреса клиента</li>
<li>аналогичную таблицу по типам выручки с указанием адреса клиента</li>
</ul>


Обратите внимание, что в процессе объединения таблиц данные не должны теряться.


In [170]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)


In [177]:
auto.merge(air, on='client_id', how='outer').merge(rzd, on='client_id', how='outer')

,client_id,auto_revenue,air_revenue,rzd_revenue
0,113,57483.0,NaN,10283.0
1,114,83.0,NaN,5774.0
2,115,912.0,81.0,981.0
3,116,4834.0,4.0,NaN
4,117,98.0,13.0,NaN
5,118,NaN,173.0,NaN
6,111,NaN,NaN,1093.0
7,112,NaN,NaN,2810.0


In [176]:
client_base.merge(auto, on='client_id', how='outer').merge(air, on='client_id', how='outer').merge(rzd, on='client_id', how='outer')

,client_id,address,auto_revenue,air_revenue,rzd_revenue
0,111,Комсомольская 4,NaN,NaN,1093.0
1,112,Энтузиастов 8а,NaN,NaN,2810.0
2,113,Левобережная 1а,57483.0,NaN,10283.0
3,114,Мира 14,83.0,NaN,5774.0
4,115,ЗЖБИиДК 1,912.0,81.0,981.0
5,116,Строителей 18,4834.0,4.0,NaN
6,117,Панфиловская 33,98.0,13.0,NaN
7,118,Мастеркова 4,NaN,173.0,NaN


### Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

<ul>
<li>У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?</li>
<li>Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?</li>
<li>Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.</li>
</ul>


### Ответ:

Координаты сессии пользователя дают нам возможность получить информацию о регионе пользователя (страна/город в зависимости от нужд можно выбрать разную детализацию). Эту информацию можно агрегировать вдоль даты в пивот где столбами будет выступать регион а потом сджоинить в уже реализованной таблицей визиты-покупки